In [7]:
import sys

if 'workhorse' not in sys.executable.split('/'):
    origin = 'workspace/'
    sys.path.append('/media/')
else:
    origin = 'data/Aldhani/eoagritwin/'
    sys.path.append('/home/potzschf/repos/')

from helperToolz.polygons_to_labels import *
from helperToolz.helpsters import *

IACS_path = f'/{origin}fields/IACS/1_Polygons/'
temp_folder = f'/{origin}fields/IACS/temp_trash/'
out_folder = f'/{origin}fields/Fine_tune/'
FORCE_folder = f'/{origin}force/output/'
aux_vrt_path = f'/{origin}fields/Auxiliary/vrt/'

# set state and year
states = ['Brandenburg', 'Niedersachsen', 'MV', 'NRW', 'Saarland']
state_folders = ['BRB', 'LSA', 'MV', 'NRW', 'SL']
state_types = ['.geoparquet','.geoparquet','.shp','.geoparquet','.shp']
state_exclude_columns = ['EC_hcat_n', 'EC_hcat_n', 'NU_BEZ', 'EC_hcat_n', 'BEZ']
state_burners = ['field_id', 'field_id', 'ID', 'field_id','LWREFSID']
years = [2022, 2024, 2023, 2020, 2021]

chip_size = 256

In [8]:
# get IACS file
for state, state_folder, state_type, state_exclude_column, state_burner, year in\
    zip(states, state_folders, state_types, state_exclude_columns, state_burners,years):
    path = [file for file in getFilelist(IACS_path + state_folder, state_type) if str(year) in file][0]
    

In [9]:
# first, make raster vrts of FORCE output that can be used ass extent raster for IACS rasterization
force_path = f'{FORCE_folder}{state_folder}/{year}/'
getFilelist(force_path, '.tif', deep=True)

reduced_files = reduce_forceTSA_output_to_validmonths(force_path, 3, 8)
ordered_files = force_order_Colors_for_VRT(reduced_files, ['BLU', 'GRN', 'RED', 'BNR'], [f'MONTH-{d:02d}' for d in range(3,9,1)])

vrt_out = path_safe(f'{aux_vrt_path}{state_folder}/{year}/')

if os.path.isdir(vrt_out):
    if len(getFilelist(f'{vrt_out}', '.vrt', deep=True)) > 0:
        print('VRT seems to be already computated, probably to create masks based on IACS')
    else:
        force_to_vrt(reduced_files, ordered_files, vrt_out, True, bandnames=['BLU', 'GRN', 'RED', 'BNR'])
else:
    os.makedirs(vrt_out)
    force_to_vrt(reduced_files, ordered_files, vrt_out, True, bandnames=['BLU', 'GRN', 'RED', 'BNR'])

vrt_cube_path = [file for file in getFilelist(vrt_out, '.vrt', deep=True) if 'Cube' in file][0]

VRT seems to be already computated, probably to create masks based on IACS


In [10]:
fields_path = f'{temp_folder}{state}/{state}_{year}_Fields.tif'
polyon_lines_path = f'{temp_folder}{state}/{state}_{year}_lines.gpkg'
borders_path = f'{temp_folder}{state}/{state}_{year}_rasterlines_touch_true.tif'
distance_path = f'{temp_folder}{state}/{state}_{year}_distance_to_border.tif'
fieldsID_path = f'{temp_folder}{state}/{state}_{year}_Field_IDs.tif'

# 1. rasterize fields (polygons) All_touch = False
make_crop_mask(path_to_polygon=path,
               path_to_extent_raster=vrt_cube_path,
               path_to_mask_out=path_safe(fields_path),
               all_touch=False, 
               categories=EXCLUDE_LIST,
               category_col=state_exclude_column) 

# 2 .rasterize borders (after polygons to lines) All_touch = True
polygons_to_lines(path_to_polygon=path,
                  path_to_lines_out=polyon_lines_path,
                  categories=EXCLUDE_LIST,
                  category_col=state_exclude_column)

# rasterize lines
rasterize_lines(path_to_lines=f'{temp_folder}{state}/{state}_{year}_lines.gpkg', 
                path_to_extent_raster=vrt_cube_path, 
                path_to_rasterlines_out=borders_path,
                all_touch=True)

# 3. distance to border raster
get_distance_raster(path_to_object=f'{temp_folder}{state}/{state}_{year}_rasterlines_touch_true.tif',
                    outPath=distance_path)

# 4. get field IDs 
make_crop_mask(path_to_polygon=path,
               path_to_extent_raster=vrt_cube_path,
               path_to_mask_out=fieldsID_path,
               all_touch=False, 
               categories=EXCLUDE_LIST,
               category_col=state_exclude_column,
               burn_col=state_burner) 

Mask for /workspace/fields/IACS/1_Polygons/SL/sa_2021.shp already exists!!!
Polylines for polygon /workspace/fields/IACS/1_Polygons/SL/sa_2021.shp already exists!!!
Rasterized lines for /workspace/fields/IACS/temp_trash/Saarland/Saarland_2021_lines.gpkg already exists!!!
distance raster for /workspace/fields/IACS/temp_trash/Saarland/Saarland_2021_rasterlines_touch_true.tif already exists!!!
Mask for /workspace/fields/IACS/1_Polygons/SL/sa_2021.shp already exists!!!


In [ ]:
# 5. make raster and label chips

# label creation and clean- up temp folder
arr_stack = stackReader(stack_tifs([fields_path, borders_path, distance_path, fieldsID_path], d_type=gdal.GDT_Float32))

In [12]:
row_col_ind =get_row_col_indices(chip_size, 0, arr_stack.shape[0], arr_stack.shape[1])

In [13]:
row_start = row_col_ind[0]
row_end   = row_col_ind[1]
col_start = row_col_ind[2]
col_end   = row_col_ind[3]

gtiff_driver = gdal.GetDriverByName('GTiff')
vrt_ds = gdal.Open(fieldsID_path)
geoTF = vrt_ds.GetGeoTransform()

In [8]:
dat = loadVRTintoNumpyAI4(f'{aux_vrt_path}{state_folder}/{year}/{getFORCExyRangeName(get_forcetiles_range(reduced_files))}')

In [ ]:
for i in range(len(row_end)):
    for j in range(len(col_end)):

        # export labels as tif chips
        out_ds = gtiff_driver.Create(path_safe(f'{out_folder}Label/{state}/{year}/{state}_{year}_{row_start[i]}_{col_start[j]}.tif'),\
                                     int(chip_size), int(chip_size), 4, gdal.GDT_Float32)
        # change the Geotransform for each chip
        geotf = list(geoTF)
        # get column and rows from filenames
        geotf[0] = geotf[0] + geotf[1] * col_start[j]
        geotf[3] = geotf[3] + geotf[5] * row_start[i]
        #print(f'X:{geoTF[0]}  Y:{geoTF[3]}  AT {file}')
        
        out_ds.SetGeoTransform(tuple(geotf))
        out_ds.SetProjection(vrt_ds.GetProjection())

        for band in range(4):
            out_ds.GetRasterBand(band + 1).WriteArray(arr_stack[row_start[i]:row_end[i], col_start[j]:col_end[j],band])
        del out_ds

        # make Sentinel-2 chips
        out_ds = gtiff_driver.Create(path_safe(f'{out_folder}img/{state}/{year}/{state}_{year}_{row_start[i]}_{col_start[j]}.tif'),\
                                     int(chip_size), int(chip_size), 4, gdal.GDT_Float32)
        
            

In [ ]:
for path in [fields_path, polyon_lines_path, borders_path, distance_path, fieldsID_path]:
    RasterKiller(path)